<a href="https://colab.research.google.com/github/tito-bot/ChatBot-NLP/blob/main/NLPprojectNov16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: give me a code so that i can mount my google drive to this file so data extraction.

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd
import json
import os

In [ ]:
import tensorflow as tf

# Get the list of available GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:
    try:
        # Set memory growth to False for each GPU
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, False)

        # Log how much memory is allocated
        print("All available GPU memory will be allocated.")
    except RuntimeError as e:
        # Memory growth must be set before initializing GPUs
        print(e)

In [ ]:
json_files = [
    "/content/drive/MyDrive/NLP_project/train.json",
    "/content/drive/MyDrive/NLP_project/test_freq.json",
    "/content/drive/MyDrive/NLP_project/test_rare.json",
    "/content/drive/MyDrive/NLP_project/valid_freq.json",
    "/content/drive/MyDrive/NLP_project/valid_rare.json"
]
data=[]

In [ ]:
def process_json(file):
    with open(file, 'r') as f:
        conversations = json.load(f)  # Load the JSON data

        # Iterate over each conversation in the file
        for conv_id, conv_data in conversations.items():
            content = conv_data['content']  # Access the conversation turns
            for turn in content:
                # Extract relevant fields from each conversation turn
                message = turn.get('message', '')
                agent = turn.get('agent', '')
                sentiment = turn.get('sentiment', '')
                turn_rating = turn.get('turn_rating', '')
                knowledge_source = turn.get('knowledge_source', [])

                # Append the structured data as a row
                data.append({
                    'message': message,
                    'agent': agent,
                    'sentiment': sentiment,
                    'turn_rating': turn_rating,
                    'knowledge_source': ', '.join(knowledge_source)  # Join list as a string
                })


In [ ]:
for json_file in json_files:
    process_json(json_file)

In [ ]:
df = pd.DataFrame(data)
df.to_csv('structured_conversations.csv', index=False)
print(df.head)

<bound method NDFrame.head of                                                   message    agent  \
0                   Are you a fan of Google or Microsoft?  agent_1   
1       Both are excellent technology they are helpful...  agent_2   
2       I'm not  a huge fan of Google, but I use it a ...  agent_1   
3       Google provides online related services and pr...  agent_2   
4       Yeah, their services are good. I'm just not a ...  agent_1   
...                                                   ...      ...   
235276  I didn't know that and I wouldn't like to be t...  agent_1   
235277  I am both terrified and amazed of them , maybe...  agent_2   
235278  lol  Sharks evolved earlier than dinosaurs and...  agent_1   
235279                well I love to talk about animals!!  agent_2   
235280                         Me too.  Have a great day!  agent_1   

                     sentiment turn_rating knowledge_source  
0       Curious to dive deeper        Good              FS1  
1    

In [ ]:
from tokenizers import Tokenizer, models, trainers, pre_tokenizers, processors
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.normalizers import NFKC
from tokenizers.processors import BertProcessing
from tokenizers.trainers import BpeTrainer
import pandas as pd

In [ ]:
df = pd.read_csv('structured_conversations.csv')
texts = df['message'].tolist()
tokenizer = Tokenizer(models.BPE())
tokenizer.normalizer = NFKC()
tokenizer.pre_tokenizer = Whitespace()
trainer = BpeTrainer(vocab_size=10000, special_tokens=["<s>", "<pad>", "</s>", "<unk>"])
tokenizer.train_from_iterator(texts, trainer=trainer)
tokenizer.post_processor = BertProcessing(("</s>", tokenizer.token_to_id("</s>")),
                                          ("<s>", tokenizer.token_to_id("<s>")))
tokenizer.save("bpe_tokenizer.json")
encoded = tokenizer.encode("Both are excellent technology they are helpful in many ways.")
print("Tokenized IDs:", encoded.ids)
print("Tokenized Tokens:", encoded.tokens)
df['tokenized_message'] = df['message'].apply(lambda x: tokenizer.encode(x).tokens)
df.to_csv('tokenized_conversations.csv', index=False)

Tokenized IDs: [0, 5467, 183, 3847, 1663, 189, 183, 5225, 126, 444, 540, 18, 2]
Tokenized Tokens: ['<s>', 'Both', 'are', 'excellent', 'technology', 'they', 'are', 'helpful', 'in', 'many', 'ways', '.', '</s>']


In [ ]:
from tokenizers import Tokenizer, models, trainers, pre_tokenizers
import pandas as pd

In [ ]:
df = pd.read_csv('structured_conversations.csv')
texts = df['message'].tolist()
tokenizer = Tokenizer(models.BPE())
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()
trainer = trainers.BpeTrainer(vocab_size=20000, special_tokens=["<s>", "<pad>", "</s>", "<unk>"])
tokenizer.train_from_iterator(texts, trainer=trainer)
tokenizer.save("bpe_tokenizer.json")

In [ ]:
import tensorflow as tf
from tokenizers import Tokenizer
import numpy as np
import pandas as pd

In [ ]:
tokenizer = Tokenizer.from_file("bpe_tokenizer.json")
df = pd.read_csv('structured_conversations.csv')
def tokenize_texts(texts, tokenizer, max_length=130):
    tokenized = []
    for text in texts:
        tokens = tokenizer.encode(text).ids
        # Truncate (post-truncation) if longer than max_length
        if len(tokens) > max_length:
            tokens = tokens[:max_length]
        tokenized.append(tokens)
    return tokenized
texts = df['message'].tolist()
tokenized_sequences = tokenize_texts(texts, tokenizer, max_length=130)
padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(
    tokenized_sequences, maxlen=130, padding='pre', truncating='post', value=tokenizer.token_to_id("<pad>")
)
attention_masks = np.where(padded_sequences != tokenizer.token_to_id("<pad>"), 1, 0)
input_sequences = tf.convert_to_tensor(padded_sequences)
attention_masks = tf.convert_to_tensor(attention_masks)
batch_size = 32  # Set your batch size
dataset = tf.data.Dataset.from_tensor_slices((input_sequences, attention_masks))
dataset = dataset.shuffle(buffer_size=len(input_sequences)).batch(batch_size)
for batch_input, batch_mask in dataset.take(1):
    print("Input batch shape:", batch_input.shape)
    print("Attention mask batch shape:", batch_mask.shape)

Input batch shape: (32, 130)
Attention mask batch shape: (32, 130)


In [ ]:
import numpy as np
import tensorflow as tf

glove_file_path = '/content/drive/MyDrive/NLP_project/glove.6B.200d.txt'
embedding_dim = 200
vocab_size = 20000  # Ensure this matches the size of your tokenizer's vocabulary
glove_embeddings_index = {}
with open(glove_file_path, 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype='float32')
        glove_embeddings_index[word] = vector

print(f"Loaded {len(glove_embeddings_index)} word vectors from GloVe.")

Loaded 400000 word vectors from GloVe.


In [ ]:
embedding_matrix = np.zeros((vocab_size, embedding_dim))
tokenizer = tf.keras.preprocessing.text.Tokenizer()
for word, index in tokenizer.word_index.items():
    if index < vocab_size:
        embedding_vector = glove_embeddings_index.get(word)
        if embedding_vector is not None:
            # Words found in GloVe will have their embeddings loaded
            embedding_matrix[index] = embedding_vector
        else:
            # Words not found in GloVe will be left as zeros or can be randomly initialized
            embedding_matrix[index] = np.random.normal(size=(embedding_dim,))

print(f"Created an embedding matrix of shape {embedding_matrix.shape}.")

Created an embedding matrix of shape (20000, 200).


In [ ]:
print(embedding_matrix)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
embedding_layer = tf.keras.layers.Embedding(
    input_dim=vocab_size,
    output_dim=embedding_dim,
    weights=[embedding_matrix],
    input_length=130,  # Maximum sequence length
    trainable=False  # Set to False if you don't want to update the GloVe embeddings during training
)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
!pip install datasets

import tensorflow as tf
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
from datasets import Dataset
import pandas as pd

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
# Load GPT-2 tokenizer from Hugging Face
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token  # Set PAD token to EOS token

# Load GPT-2 model
model = TFGPT2LMHeadModel.from_pretrained('gpt2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [ ]:
# Maximum sequence length and batch size
MAX_SEQ_LENGTH = 130
BATCH_SIZE = 8

In [ ]:
# Function to tokenize and encode input text
def tokenize_function(examples):
    return tokenizer(
        examples["message"],
        padding="max_length",
        truncation=True,
        max_length=MAX_SEQ_LENGTH,
        return_tensors='tf',  # Ensure to return tensors for TensorFlow
    )

In [ ]:
# Load your dataset (assuming it's saved as a CSV file)
df = pd.read_csv('/content/drive/MyDrive/NLP_project/structured_conversations.csv')

# Convert the DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Tokenize the dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=['agent', 'sentiment', 'turn_rating', 'knowledge_source'])

Map:   0%|          | 0/235281 [00:00<?, ? examples/s]

In [ ]:
# Convert tokenized dataset to TensorFlow format
def generate_tf_dataset(tokenized_data, batch_size, shuffle=True):
    input_ids = tokenized_data['input_ids']
    attention_masks = tokenized_data['attention_mask']

    dataset = tf.data.Dataset.from_tensor_slices(({
        'input_ids': input_ids, 'attention_mask': attention_masks
    }, input_ids))  # Model needs inputs and labels (here labels are same as input_ids)

    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(input_ids))

    dataset = dataset.batch(batch_size)
    return dataset

In [ ]:
# Create TensorFlow dataset for training
tf_dataset = generate_tf_dataset(tokenized_dataset, BATCH_SIZE)

In [ ]:
import tensorflow as tf

# Define loss manually if compute_loss is causing issues
def custom_loss(y_true, y_pred):
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    return loss_fn(y_true, y_pred)

# Use 'adam' as the optimizer identifier instead of the object
#model.compile(optimizer=optimizer, loss=custom_loss)
model.compile(optimizer='adam', loss=custom_loss)

# Ensure the dataset includes both inputs and labels
history = model.fit(
    tf_dataset,
    epochs=20,
    verbose=1
)

Epoch 1/20
   55/29411 [..............................] - ETA: 212:24:11 - loss: 0.4441